In [18]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
[] != []

False

In [22]:
import numpy as np
test = np.array([["erste, wohnung"], ["ja"]])
print(test.flatten())
print(test)
res = np.array([])
for x in range(test.size):
    res = np.append(res,np.array(test[x]).flatten())
    #print(res.shape)
    res_2 = np.array([])
    for x in range(res.size):
        res_2 = np.append(res_2,np.array(res[x]).flatten())
        print(res_2.shape, res_2)
    
#print(np.array([["erste, wohnung"], ["ja"]]).flatten().size)
#def rek_flatten(a):
#    for x in range(a.size):
#    res = np.append(res,np.array(test[x]).flatten())
#    #print(res.shape)
#    res_2 = np.array([])

['erste, wohnung' 'ja']
[['erste, wohnung']
 ['ja']]
(1,) ['erste, wohnung']
(1,) ['erste, wohnung']
(2,) ['erste, wohnung' 'ja']


In [4]:
import os

import spacy
from IPython.display import display

from src.pipelinelib.querying import Parser, Queryable

folder = os.path.join(os.getcwd(), "data", "transcripts")
files = [os.path.join(root, f) for root, _, files in os.walk(folder)
         for f in files if f.endswith(".docx")]


nlp = spacy.load("de_core_news_sm", disable=["ner", "parser"])
parser = Parser(
    nlp=nlp, metadata_path="./data/transcripts/Kopie von Transkriptionspaare_Daten.xls")

parser.read_from_files(files)
display(parser.frame)


=== Parser: reading from /home/rise/Schreibtisch/Sigmund_git/sigmund/data/transcripts/Paar 81_T1_IM_FW.docx ===
=== Parser: reading from /home/rise/Schreibtisch/Sigmund_git/sigmund/data/transcripts/Paar 87_T1_IM_FW.docx ===
=== Parser: reading from /home/rise/Schreibtisch/Sigmund_git/sigmund/data/transcripts/Paar 182_T1_IM_FW.docx ===
=== Parser: reading from /home/rise/Schreibtisch/Sigmund_git/sigmund/data/transcripts/Paar 138_T1_IM_FW.docx ===
=== Parser: reading from /home/rise/Schreibtisch/Sigmund_git/sigmund/data/transcripts/Paar 47_T1_IM_FW.docx ===
=== Parser: reading from /home/rise/Schreibtisch/Sigmund_git/sigmund/data/transcripts/Paar 105_T1_IM_FW.docx ===
=== Parser: reading from /home/rise/Schreibtisch/Sigmund_git/sigmund/data/transcripts/Paar 29_T1_IM_FW.docx ===
=== Parser: reading from /home/rise/Schreibtisch/Sigmund_git/sigmund/data/transcripts/Paar 27_T1_IM_FW.docx ===
=== Parser: reading from /home/rise/Schreibtisch/Sigmund_git/sigmund/data/transcripts/Paar 58_T1_IM_F

,document_id,paragraph_id,sentence_id,couple_id,speaker,gender,is_depressed_group,depressed_person,text
0,0,0,0,81,A,W,False,None,Ich dachte wir reden darüber wie wir die erste...
1,0,1,1,81,B,M,False,None,Erste Wohnung?
2,0,2,2,81,A,W,False,None,"Ja, ich dachte das ist ein nettes Thema, oder?"
3,0,3,3,81,B,M,False,None,Ja.
4,0,4,4,81,A,W,False,None,"Ich fand das süß, wie du erst angefangen hast ..."
...,...,...,...,...,...,...,...,...,...
126,9,94,126,60,B,M,True,W,Hab ich ein Schild auf dem Rücken?
127,9,94,127,60,B,M,True,W,Ladet eure Probleme bei mir ab.
128,9,95,128,60,A,W,True,W,Das sind keine Probleme.
129,9,96,129,60,B,M,True,W,Kümmerst du dich drum.


In [5]:
from src.pipelinelib.querying import Parser, Queryable
from src.pipelinelib.text_body import TextBody
from src.pipelinelib.pipeline import Pipeline
queryable = Queryable.from_parser(parser)

In [23]:
from src.sigmund import adapter
from src.sigmund.extensions import LEMMATIZED,TALKTURN

from src.sigmund.features import agreement_score as fagree
from src.sigmund.features import flesch_reading_ease as fflesch
from src.sigmund.features import talk_turn as ftalkturn
from src.sigmund.features import liwc_one_hot as fliwconehot
from src.sigmund.preprocessing import words as pwords

liwc_dict_path = r"/home/rise/Schreibtisch/Sigmund_git/sigmund/data/German_LIWC2001_Dictionary.dic"

pipeline = Pipeline(queryable = queryable, empty_pipeline=True) \
    .add_component(pwords.Tokenizer())\
    .add_component(fliwconehot.LiwcOneHot())
    #.add_component(ftalkturn.TalkTurnExtractor())\
    #.add_component(fagree.AgreementScoreExtractor())\
    #.add_component(fflesch.FleschExtractor())

    #.add_component(fliwconehot.LiwcOneHot())
    #.add_component(adapter.Adapter(old = TOKENS_sese, new = TALKTURN))\


In [28]:
test = pipeline.execute()

#for doc in preprocessed_depr:
#    print('TT',doc[0]._.talkturn, 'AS',doc[0]._.agreementscore)
#for doc in preprocessed_non_depr:
#    print('TT',doc[0]._.talkturn, 'AS',doc[0]._.agreementscore)

=== Starting pipeline with ['Tokenizer', 'LiwcOneHot'] ===
Executing Tokenizer
=== Queryable is executing on Document level, query = '' ===


document_id                                                    5
couple_id                                                    105
text           [ , jetzt, sollen, wir, uns, unterhalten, gena...
Name: 5, dtype: object

=== Queryable is executing on Paragraph level, query = '' ===
=== Queryable is executing on Sentence level, query = '' ===
Executing LiwcOneHot


,document_id,couple_id,paragraph_id,speaker,text
0,0,81,0,A,"[ich, dachte, wir, reden, darüber, wie, wir, d..."
1,0,81,1,B,"[erste, wohnung]"
2,0,81,2,A,"[ja, ich, dachte, das, ist, ein, nettes, thema..."
3,0,81,3,B,[ja]
4,0,81,4,A,"[ich, fand, das, süß, wie, du, erst, angefange..."
...,...,...,...,...,...
744,9,60,92,B,"[jaaa, klar, …]"
745,9,60,93,A,"[die, brauchen, neue, reifen]"
746,9,60,94,B,"[hab, ich, ein, schild, auf, dem, rücken, lade..."
747,9,60,95,A,"[das, sind, keine, probleme]"


ValueError: ('Lengths must match to compare', (749,), (0,))

In [ ]:
display(test)

In [ ]:
document_df = queryable.execute(level=TextBody.DOCUMENT)
display(document_df)

In [ ]:
paragraph_df = queryable.execute(level=TextBody.PARAGRAPH)
display(paragraph_df)

In [ ]:
sentence_df = queryable.execute(level=TextBody.SENTENCE)
display(sentence_df)

queryable.execute(level=TextBody.SENTENCE)

In [ ]:
pgs = queryable.by_couple_id(couple_id=27) \
    .is_depressed(d=False) \
    .execute(level=TextBody.DOCUMENT)
display(pgs)